## Ejercicio 8: Langchain
En este ejercicio, vemos un par de ejemplos con LangChain. Primero, refinaremos un prompt, y después lo usaremos para crear un resumen

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

print(type(OPENAI_API_KEY),OPENAI_API_KEY)
print(type(API_VERSION),API_VERSION)
print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [2]:
import os

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Importamos langchain
Si no está instalado, descomenta la próxima línea

In [ ]:
!pip -q install langchain
!pip -q install langchain-openai

##### Primero, definimos dos LLMs, uno para refinar un prompt (tarea creativa) y el segundo para utilizar dicho prompt (tarea poco creativa)

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_ENDPOINT

llm_refine = AzureChatOpenAI(azure_deployment="gpt-4o",
    api_key = OPENAI_API_KEY,
    api_version=API_VERSION,
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

llm_summary = AzureChatOpenAI(azure_deployment="gpt-4o",
    api_key = OPENAI_API_KEY,
    api_version=API_VERSION,
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

##### Definimos un prompt base para generar resúmenes, y su versión para ser refinado

In [6]:
base_summary_prompt = """
Eres un asistente experto en síntesis de información.
Tu tarea es leer el siguiente texto y generar un resumen que destaque las ideas principales.
Asegúrate de:
- Identificar los puntos clave.
- Omitir detalles secundarios.
- Mantener la coherencia y cohesión en el resumen.

Texto:
{text}

Resumen:"""


refinement_template = """
Eres un experto en ingeniería de prompts.
Reformula, mejora y amplía con mucho más detalle el siguiente prompt de resumen para que sea más claro, específico y efectivo. El texto a resumir será del dominio de las finanzas y banca:
Prompt original:
{base_prompt}

Devuelve UNICAMENTE el prompt refinado, sin más texto antes o después.

Prompt refinado:
"""

##### Ahora usamos el módulo PromptTemplate de Langchain, y LLMChain para usar programáticamente dicho template con el LLM que hemos definido antes

Hasta que no ejecutamos el comando "run", no está llamando al LLM, solo definiendo las instrucciones (como en Spark)

In [ ]:
from IPython.display import Markdown
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain.*")

refine_prompt_template = PromptTemplate(
    template=refinement_template,
    input_variables=["base_prompt"]
)
refine_chain = LLMChain(llm=llm_refine, prompt=refine_prompt_template)

refined_prompt = refine_chain.run(base_prompt=base_summary_prompt)
print("Prompt refinado:\n")
display(Markdown(refined_prompt))

In [8]:
refine_prompt_template

PromptTemplate(input_variables=['base_prompt'], input_types={}, partial_variables={}, template='\nEres un experto en ingeniería de prompts.\nReformula, mejora y amplía con mucho más detalle el siguiente prompt de resumen para que sea más claro, específico y efectivo. El texto a resumir será del dominio de las finanzas y banca:\nPrompt original:\n{base_prompt}\n\nDevuelve UNICAMENTE el prompt refinado, sin más texto antes o después.\n\nPrompt refinado:\n')

In [ ]:
from IPython.display import Markdown
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain.*")


# Crear el prompt template
refine_prompt_template = PromptTemplate(
   template=refinement_template,
   input_variables=["base_prompt"]
)


# Crear la cadena LLM con tu modelo
refine_chain = LLMChain(llm=llm_refine, prompt=refine_prompt_template)


# Ejecutar correctamente con invoke (nuevo estándar porque en versiones anteriores run() esta deprecado)
refined_prompt = refine_chain.invoke({"base_prompt": base_summary_prompt})


# Inspeccionar o extraer resultado (ajusta si la clave es diferente)
refined_prompt_str = refined_prompt["text"] if "text" in refined_prompt else refined_prompt["base_prompt"]


# Mostrar el resultado con estilo Markdown
print("Prompt refinado:\n")
display(Markdown(refined_prompt_str))


##### Ahora, creamos la cadena de resumen usando el prompt refinado

In [ ]:
refined_summary_prompt_template = PromptTemplate(
    template=refined_prompt_str,
    input_variables=["text"]
)
summary_chain = LLMChain(llm=llm_summary, prompt=refined_summary_prompt_template)

with open("financial_report.txt", "r", encoding="utf-8") as f:
    texto_extenso = f.read()

resumen = summary_chain.run(text=texto_extenso)
print("\nResumen generado:\n")
display(Markdown(resumen))

In [ ]:
from IPython.display import Markdown
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain.*")


# Crear el prompt template
refine_prompt_template = PromptTemplate(
   template=refinement_template,
   input_variables=["base_prompt"]
)


# Crear la cadena LLM con tu modelo
refine_chain = LLMChain(llm=llm_refine, prompt=refine_prompt_template)


# Ejecutar correctamente con invoke (nuevo estándar porque en versiones anteriores run() esta deprecado)
refined_prompt = refine_chain.invoke({"base_prompt": base_summary_prompt})


# Inspeccionar o extraer resultado (ajusta si la clave es diferente)
refined_prompt_str = refined_prompt["text"] if "text" in refined_prompt else refined_prompt["base_prompt"]


# Mostrar el resultado con estilo Markdown
print("Prompt refinado:\n")
display(Markdown(refined_prompt_str))


- ¿Qué ventajas le ves a usar LangChain en vez de hacer todo a mano?
- ¿Te gusta más este resumen o el que hemos hecho antes? ¿Y si cambiamos la temperatura o el modelo?

In [ ]:
refined_summary_prompt_template = PromptTemplate(
    template=refined_prompt["text"],
    input_variables=["text"]
)
summary_chain = LLMChain(llm=llm_summary, prompt=refined_summary_prompt_template)

with open("financial_report.txt", "r", encoding="utf-8") as f:
    texto_extenso = f.read()

resumen = summary_chain.run(text=texto_extenso)
print("\nResumen generado:\n")
display(Markdown(resumen))